# Logistic Regression 

In [1]:
import tenseal as ts
import pandas as pd
import random
import torch
from torch import nn
from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Dataset

Import the dataset from Kaggle, you can download the data set [here](https://www.kaggle.com/datasets/naveengowda16/logistic-regression-heart-disease-prediction?resource=download). This data set contains data by WHO regarding heart disease. 

World Health Organization has estimated 12 million deaths occur worldwide, every year due to Heart diseases. Half the deaths in the United States and other developed countries are due to cardio vascular diseases. The early prognosis of cardiovascular diseases can aid in making decisions on lifestyle changes in high risk patients and in turn reduce the complications. This research intends to pinpoint the most relevant/risk factors of heart disease as well as predict the overall risk using logistic regression.

This dataset includes patients' information along with a 10-year risk of future coronary heart disease (CHD) as a label. The goal is to build a model that can predict this 10-year CHD risk based on patients' information

In [2]:
df = pd.read_csv("../data/raw/framingham_heart_disease.csv")
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


## EDA

In [3]:
df.shape

(4238, 16)

In [4]:
df.describe()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
count,4238.000000,4238.000000,4133.000000,4238.000000,4209.000000,4185.000000,4238.000000,4238.000000,4238.000000,4188.000000,4238.000000,4238.000000,4219.000000,4237.000000,3850.000000,4238.000000
mean,0.429212,49.584946,1.978950,0.494101,9.003089,0.029630,0.005899,0.310524,0.025720,236.721585,132.352407,82.893464,25.802008,75.878924,81.966753,0.151958
std,0.495022,8.572160,1.019791,0.500024,11.920094,0.169584,0.076587,0.462763,0.158316,44.590334,22.038097,11.910850,4.080111,12.026596,23.959998,0.359023
min,0.000000,32.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,107.000000,83.500000,48.000000,15.540000,44.000000,40.000000,0.000000
25%,0.000000,42.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,206.000000,117.000000,75.000000,23.070000,68.000000,71.000000,0.000000
50%,0.000000,49.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,234.000000,128.000000,82.000000,25.400000,75.000000,78.000000,0.000000
75%,1.000000,56.000000,3.000000,1.000000,20.000000,0.000000,0.000000,1.000000,0.000000,263.000000,144.000000,89.875000,28.040000,83.000000,87.000000,0.000000
max,1.000000,70.000000,4.000000,1.000000,70.000000,1.000000,1.000000,1.000000,1.000000,696.000000,295.000000,142.500000,56.800000,143.000000,394.000000,1.000000


In [5]:
# check for any missing values
df.isna().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [6]:
def clean_dataset(df):

    # We could either populate the missing values or just
    # remove the entire column if not needed any more.

    df["education"] = df["education"].fillna(df["education"].mean())
    df["cigsPerDay"] = df["cigsPerDay"].fillna(df["cigsPerDay"].mean())
    df["BPMeds"] = df["BPMeds"].fillna(df["BPMeds"].mean())
    df["totChol"] = df["totChol"].fillna(df["totChol"].mean())
    df["BMI"] = df["BMI"].fillna(df["BMI"].mean())
    df["heartRate"] = df["heartRate"].fillna(df["heartRate"].mean())
    df["glucose"] = df["glucose"].fillna(df["glucose"].mean())

    # remove rows with missing values
    df = df.dropna()

    new_df = df.groupby("TenYearCHD")
    df = new_df.apply(lambda x: x.sample(new_df.size().min(), random_state=73).reset_index(drop=True))
    return df

In [7]:
clean_dataset(df)

male  age  education  currentSmoker  cigsPerDay  BPMeds  \
TenYearCHD                                                                
0          0       1   44        3.0              0         0.0     0.0   
           1       0   60        1.0              0         0.0     0.0   
           2       1   45        1.0              1        30.0     0.0   
           3       1   47        1.0              1        30.0     0.0   
           4       0   43        4.0              1         9.0     0.0   
...              ...  ...        ...            ...         ...     ...   
1          639     1   62        1.0              1        43.0     0.0   
           640     1   37        1.0              1        30.0     0.0   
           641     1   59        1.0              1         3.0     0.0   
           642     0   58        3.0              0         0.0     0.0   
           643     1   51        1.0              1        20.0     0.0   

                prevalentStroke  prevalentHyp  diabetes  totChol  sysBP  \
TenYearCHD                                                                
0          0                  0             0         0    243.0  146.0   
           1                  0             0         0    236.0  126.0   
           2                  0             0         0    240.0  141.0   
           3                  0             1         0    190.0  147.5   
           4                  0             0         0    207.0   95.5   
...                         ...           ...       ...      ...    ...   
1          639                0             0         0    217.0  107.5   
           640                0             0         0    179.0  125.0   
           641                0             1         1    230.0  182.0   
           642                0             1         0    241.0  143.5   
           643                0             1         0    239.0  168.0   

                diaBP    BMI  heartRate     glucose  TenYearCHD  
TenYearCHD                                                       
0          0     91.0  26.72       80.0  104.000000           0  
           1     84.0  20.34       71.0   76.000000           0  
           2     89.0  25.01       95.0   76.000000           0  
           3     92.5  31.31       77.0   82.000000           0  
           4     70.0  19.78       93.0   79.000000           0  
...               ...    ...        ...         ...         ...  
1          639   75.0  26.21       80.0   66.000000           1  
           640   82.0  19.53       60.0   70.000000           1  
           641  102.0  25.91       66.0  147.000000           1  
           642   85.5  23.96       96.0   81.966753           1  
           643  102.0  30.38       82.0   68.000000           1  

[1288 rows x 16 columns]

In [8]:
# check if any null or na values exist
df.isna().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [9]:
# Fetch X and y
y = df["TenYearCHD"].values
X = df.drop("TenYearCHD", axis = 1)

In [10]:
 X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 0,test_size = 0.2)

In [11]:
print(f"X_train has shape: {X_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"X_test has shape: {X_test.shape}")
print(f"y_test has shape: {y_test.shape}")

X_train has shape: (3390, 15)
y_train has shape: (3390,)
X_test has shape: (848, 15)
y_test has shape: (848,)


## Train a Logistic Regression Model using plaintext data

We will start by training a logistic regression model (without any encryption), which can be viewed as a single layer neural network with a single node. We will be using this model as a means of comparison against encrypted training and evaluation.

In [12]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)

/opt/app-root/lib64/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
print("Accuracy : ", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy :  0.8360849056603774
              precision    recall  f1-score   support

           0       0.84      0.99      0.91       710
           1       0.43      0.02      0.04       138

    accuracy                           0.84       848
   macro avg       0.63      0.51      0.48       848
weighted avg       0.77      0.84      0.77       848



## Train a Logistic Regression Model 
We wil now train a Logistic Regression with a single neural network with a single node. 

[Reference](https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be)

#### Preprocess data for torch model

In [15]:
torch.random.manual_seed(73)
random.seed(73)


# Split the dataset into train(80%) and test(20%)
def split_train_and_test(X, y, ratio=0.2):
    idxs = [i for i in range(len(X))]
    random.shuffle(idxs)
    # delimiter between test and train data
    delim = int(len(X) * ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return X[train_idxs], y[train_idxs], X[test_idxs], y[test_idxs]


def standardize_data():
    global df
    clean_dataset(df)
    df = (df - df.mean()) / df.std()
    X = torch.tensor(df.values).float()
    y = torch.tensor(df["TenYearCHD"].values).float().unsqueeze(1)
    df = df.drop("TenYearCHD", 'columns')
    return split_train_and_test(X, y)


def randomize_data(m=1024, n=2):
    # data separable by the line `y = x`
    X_train = torch.randn(m, n)
    X_test = torch.randn(m // 2, n)
    y_train = (X_train[:, 0] >= X_train[:, 1]).float().unsqueeze(0).t()
    y_test = (x_test[:, 0] >= X_test[:, 1]).float().unsqueeze(0).t()
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = standardize_data()


print(f"X_train has shape: {X_train.shape}")
print(f"y_train has shape: {y_train.shape}")
print(f"X_test has shape: {X_test.shape}")
print(f"y_test has shape: {y_test.shape}")

X_train has shape: torch.Size([3391, 16])
y_train has shape: torch.Size([3391, 1])
X_test has shape: torch.Size([847, 16])
y_test has shape: torch.Size([847, 1])


In [16]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)

    def forward(self, x):
        output = torch.sigmoid(self.lr(x))
        return output

In [17]:
n_features = X_train.shape[1]
model = LogisticRegression(n_features)

In [18]:
# Gradient descent with learning rate of 1
optim = torch.optim.SGD(model.parameters(), lr=1)
bceloss = torch.nn.BCELoss()

In [19]:
epochs = 3

In [20]:
def train_model(model, optim, bceloss, X, y, epochs):
    for ep in range(1, epochs+1):
        optim.zero_grad()
        result = model(X)
        loss = bceloss(result, y)
        loss.backward()
        optim.step()
        print(f"Loss at epoch number {ep}: {loss.data}")
    return model

In [21]:
model = train_model(model, optim, bceloss, X_train, y_train, epochs)

Loss at epoch number 1: 0.8681004047393799
Loss at epoch number 2: -0.4236203134059906
Loss at epoch number 3: -1.2133352756500244


In [22]:
def accuracy(model, X, y):
    out = model(X)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

In [23]:
p_accuracy = accuracy(model, X_test, y_test)
print(f"Accuracy on plain test_set using a Logistic Regression with a single neural network with a single node. : {p_accuracy}")

Accuracy on plain test_set using a Logistic Regression with a single neural network with a single node. : 0.43565526604652405


## Encrypted Evaluation

Now, we shall look into creating a Logistic Regression model with **plain parameters** at the moment, we can also use encrypted parameters but would try that in later steps. And we will evaluate this model on an encrypted test data set. 

Let's start by creating a pytorch LR model that works for encrypted data,

In [24]:
# Author : Tenseal - https://github.com/OpenMined/

class EncryptedLR:

    def __init__(self, torch_lr):
        # TenSEAL processes lists and not torch tensors,
        # so we take out the parameters from the PyTorch model
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()

    def forward(self, enc_x):
        # We don't need to perform sigmoid as this model
        # will only be used for evaluation, and the label
        # can be deduced without applying sigmoid
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)

    def decrypt(self, context):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()

In [25]:
eelr = EncryptedLR(model)

Let's create a tenseal context and specify the Homomorphic Encrytion scheme and the define the parameters we want to use. 

We will choose small and secure parameters that will enable us to perform single multiplication. This should be adequate for evaluating the Logistic Regression model. But for training the model on encrypted data, we would require larger parameters

In [26]:
# Define parameters
poly_mod_degree = 4096
coeff_mod_bit_sizes = [40, 20, 40]

In [27]:
# Create TenSEALContext
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)

In [28]:
# Scale of ciphertext to use
ctx_eval.global_scale = 2 ** 20

In [29]:
# This key is needed for doing dot-product operations
ctx_eval.generate_galois_keys()

Let's encrypt the test data set before the evaluation process. 

In [30]:
t_start = time()
enc_X_test = [ts.ckks_vector(ctx_eval, x.tolist()) for x in X_test]
t_end = time()
print(f"Encryption of the test-set took {int(t_end - t_start)} second(s)")

Encryption of the test-set took 1 second(s)


In [31]:
# encrypt the model's parameters
eelr.encrypt(ctx_eval)

We don't need the sigmoid function on the encrypted output layer, however we would need to add this to the encrypted model training. 

In [32]:
def encrypted_evaluation(model, enc_X_test, y_test):
    t_start = time()

    correct = 0
    for enc_X, y in zip(enc_X_test, y_test):
        # encrypted evaluation
        enc_out = model(enc_X)
        # plain comparaison
        out = enc_out.decrypt()
        out = torch.tensor(out)
        out = torch.sigmoid(out)
        if torch.abs(out - y) < 0.5:
            correct += 1

    t_end = time()
    print(f"Evaluated test_set of {len(X_test)} entries in {int(t_end - t_start)} seconds")
    print(f"Accuracy: {correct}/{len(X_test)} = {correct / len(X_test)}")
    return correct / len(X_test)

In [33]:
encrypted_accuracy = encrypted_evaluation(eelr, enc_X_test, y_test)
diff_accuracy = p_accuracy - encrypted_accuracy
print(f"Difference between plain and encrypted accuracies: {diff_accuracy}")
if diff_accuracy < 0:
    print("Looks like we got a better accuracy on the encrypted test-set! The noise was on our end ")

Evaluated test_set of 847 entries in 2 seconds
Accuracy: 133/847 = 0.15702479338842976
Difference between plain and encrypted accuracies: 0.27863046526908875


## Training an Encrypted Logistic Regression model on Encrypted Data

Now, we will redefine a PyTorch model that can both forward encrypted data, as well as backpropagate to update the weights and thus train the encrypted logistic regression model on encrypted data. 

In [34]:
# Author : Tenseal - https://github.com/OpenMined/
class EncryptedLR:

    def __init__(self, torch_lr):
        self.weight = torch_lr.lr.weight.data.tolist()[0]
        self.bias = torch_lr.lr.bias.data.tolist()
        # we accumulate gradients and counts the number of iterations
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0

    def forward(self, enc_x):
        enc_out = enc_x.dot(self.weight) + self.bias
        enc_out = EncryptedLR.sigmoid(enc_out)
        return enc_out
    
    def backward(self, enc_x, enc_out, enc_y):
        out_minus_y = (enc_out - enc_y)
        self._delta_w += enc_x * out_minus_y
        self._delta_b += out_minus_y
        self._count += 1

    def update_parameters(self):
        if self._count == 0:
            raise RuntimeError("You should at least run one forward iteration")
        # update weights
        # We use a small regularization term to keep the output
        # of the linear layer in the range of the sigmoid approximation
        self.weight -= self._delta_w * (1 / self._count) + self.weight * 0.05
        self.bias -= self._delta_b * (1 / self._count)
        # reset gradient accumulators and iterations count
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0

    @staticmethod
    def sigmoid(enc_x):
        # We use the polynomial approximation of degree 3
        # sigmoid(x) = 0.5 + 0.197 * x - 0.004 * x^3
        # from https://eprint.iacr.org/2018/462.pdf
        # which fits the function pretty well in the range [-5,5]
        return enc_x.polyval([0.5, 0.197, 0, -0.004])

    def plain_accuracy(self, x_test, y_test):
        # evaluate accuracy of the model on
        # the plain (x_test, y_test) dataset
        w = torch.tensor(self.weight)
        b = torch.tensor(self.bias)
        out = torch.sigmoid(x_test.matmul(w) + b).reshape(-1, 1)
        correct = torch.abs(y_test - out) < 0.5
        return correct.float().mean()
    
    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)

    def decrypt(self):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [35]:
# Specify parameters
poly_mod_degree = 8192
coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]

In [36]:
# create TenSEALContext
ctx_training = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_training.global_scale = 2 ** 21
ctx_training.generate_galois_keys()

In [37]:
t_start = time()
enc_X_train = [ts.ckks_vector(ctx_training, x.tolist()) for x in X_train]
enc_y_train = [ts.ckks_vector(ctx_training, y.tolist()) for y in y_train]
t_end = time()
print(f"Encryption of the training_set took {int(t_end - t_start)} seconds")

Encryption of the training_set took 66 seconds


We finally reached the last part, which is about training an encrypted logistic regression model on encrypted data! You can see that we decrypt the weights and re-encrypt them again after every epoch, this is necessary since after updating the weights at the end of the epoch, we can no longer use them to perform enough multiplications, so we need to get them back to the initial ciphertext level. In a real scenario, this would translate to sending the weights back to the secret-key holder for decryption and re-encryption. In that case, it will result in just a few Kilobytes of communication per epoch.

In [38]:
eelr = EncryptedLR(LogisticRegression(n_features))
accuracy = eelr.plain_accuracy(X_test, y_test)
print(f"Accuracy at epoch 0 is {accuracy}")

times = []
for epoch in range(epochs):
    eelr.encrypt(ctx_training)
    
    # if you want to keep an eye on the distribution to make sure
    # the function approxiamation is still working fine
    # WARNING: this operation is time consuming
    # encrypted_out_distribution(eelr, enc_X_train)
    
    t_start = time()
    for enc_X, enc_y in zip(enc_X_train, enc_y_train):
        enc_out = eelr.forward(enc_X)
        eelr.backward(enc_X, enc_out, enc_y)
    eelr.update_parameters()
    t_end = time()
    times.append(t_end - t_start)
    
    eelr.decrypt()
    accuracy = eelr.plain_accuracy(X_test, y_test)
    print(f"Accuracy at epoch #{epoch + 1} is {accuracy}")


print(f"\nAverage time per epoch: {int(sum(times) / len(times))} seconds")
print(f"Final accuracy is {accuracy}")

diff_accuracy = p_accuracy - accuracy
print(f"Difference between plain and encrypted accuracies: {diff_accuracy}")
if diff_accuracy < 0:
    print("Oh! We got a better accuracy when training on encrypted data!")

Accuracy at epoch 0 is 0.0
Accuracy at epoch #1 is 0.36009445786476135
Accuracy at epoch #2 is 0.5938606858253479
Accuracy at epoch #3 is 0.5773317813873291

Average time per epoch: 350 seconds
Final accuracy is 0.5773317813873291
Difference between plain and encrypted accuracies: -0.14167651534080505
Oh! We got a better accuracy when training on encrypted data!


## Conclusion :

In this notebook, 
* We explored heart disease data set we found on Kaggle. 
* We performed EDA and created a simple Logistic Regression model on that. 
* Then we created a single layer neural network with a single node. 
* Evaluate this model on an encrypted test data set.
* Train an Encrypted Logistic Regression model on Encrypted Data.
* Accuracy is low on the training the model on encrypted data and that's because we have lessened the number of epochs to use less resources. But we can get 70-80% accuracy when trained with more epochs. 

As a part of the future work, we aim to explore more datasets and more machine learning problems that can benefit with the encrypted training and testing. 